In [1]:
import pandas as pd
from unidecode import unidecode
import Levenshtein as lev


# Анализ авторынка Казахстана

Описание проекта: Целью проекта является анализ авторынка Республики Казахстан.

Описание данных: В нашем распоряжении "сырые" данные по продажам автомобилей в Казахстане за 2019 год. Данные получены из
официальной статистики VAG, после перевода из эксель в csv обнаружились множественные
проблемы с исходными данными: некорректные разделители десятичных разрядов, несоответствие
данных типу данных. Дополнительной проблемой является то, что статистику собирал не один
человек, поэтому есть неявные дубликаты, а также одни и те же
признаки могут быть записаны как на русском, так и на английском языке.
План работы:
1. Очистка данных  
    1.1 Правильно загрузить данные с нужным разделителем десятичных разрядов;  
    1.2 Привести данные в столбцах к корректным типам;  
    1.3 Избавиться от лишних столбцов;  
    1.4 Избавиться от неявных дубликатов в столбцах (4-WD и 4WD и так далее);  
    1.5 Решить что сделать с пропусками данных (оставить или удалить);  
    1.6 Разобраться с аномалиями (100 машин в одном заказе и ягуар с двигателем 200 литров);  
    1.7 Добавить категориальные столбцы.  
2. Исследовательский анализ:  
    2.1 Столбца (расписать смысл каждого столбца. Оценить важность каждого столбца для цели задачи. Выделить группы связанных.)  
    2.2 Анализ строк (Упорядочены ли строки: по индексу/дате/признаку/что-то_ещё. Есть ли мутлтииндексы.)  
    2.3 Анализ явных дубликатов.  
    2.4 Анализ пропусков    
    2.5 Анализ типов данных. Разделить на числовые категориальные (упорядоченные/неупорядоченные), текстовые, временные.  
    2.6 Изменение типа данных и кодирование переменных.
    2.7 Анализ числовых признаков.  
    2.8 Анализ категориальных признаков.
3. Анализ рынка   
    3.1 Топовые марки на рынке Казахстана.  
    3.2 Продажи на рынке Казахстана всех марок в денежном и натуральном выражении.  
    3.3 Динамика продаж в целом по рынку (помесячно).
    3.4 Динамика продаж по категориям (помесячно): тип топлива, класс, сегмент.  
    3.5 Продажи по регионам.  
    3.6 Продажи по автоцентрам. 

## Очистка данных

In [2]:
#загрузим датасет
file_path = 'C:\\Users\\Redmi\\Desktop\\авторынок  казахстана\\auto_kz_2019.csv'
df = pd.read_csv(file_path, sep = ';',decimal=",")
print(df.head())

    Год   Месяц     Компания Бренд Модель Модификация Год выпуска  \
0  2019     Май  Mercur Auto  Audi     A3        TFSI        2018   
1  2019  Август  Mercur Auto  Audi     A3        TFSI        2018   
2  2019  Апрель  Mercur Auto  Audi     A4        TFSI        2018   
3  2019    Июль  Mercur Auto  Audi     A4        TFSI        2018   
4  2019    Июль  Mercur Auto  Audi     A4        TFSI        2018   

  Страна-производитель Вид топлива Объём двиг, л,  ... Тип клиента  \
0             Германия      Бензин            1,4  ...   Физ. Лицо   
1             Германия      Бензин            1,4  ...    Юр. Лицо   
2             Германия      Бензин            1,4  ...   Физ. Лицо   
3             Германия      Бензин            1,4  ...    Юр. Лицо   
4             Германия      Бензин            1,4  ...   Физ. Лицо   

  Форма расчета Количество Цена, USD Продажа, USD   Область  \
0   безналичный        1.0  28115.00     28115.00  г.Алматы   
1      наличный        1.0  32246.99  

In [3]:
# Приводим названия столбцов к нижнему регистру
df.columns = df.columns.str.lower()
# Заменяем пробелы и дефисы на нижние подчеркивания
df.columns = df.columns.str.replace('[ \-]', '_', regex=True)
# Заменяем пробелы с запятыми и запятые на нижние подчеркивания
df.columns = df.columns.str.replace('[ ,\,]', '', regex=True)

In [4]:
translation_dict = {
    'год': 'year',
    'месяц': 'month',
    'компания': 'company',
    'бренд': 'brand',
    'модель': 'model',
    'модификация': 'modification',
    'год_выпуска': 'manufacturing_year',
    'страна_производитель': 'manufacturer_country',
    'вид_топлива': 'fuel_type',
    'объём_двиг_л': 'engine_capacity_liters',
    'коробка_передач': 'transmission_type',
    'тип_привода': 'drive_type',
    'сегмент': 'segment',
    'регион': 'region',
    'наименование_дилерского_центра': 'dealer_center_name',
    'тип_клиента': 'customer_type',
    'форма_расчета': 'payment_form',
    'количество': 'quantity',
    'цена_usd': 'price_usd',
    'продажа_usd': 'sale_usd',
    'область': 'area',
    'сегментация_2013': 'segmentation_2013',
    'класс_2013': 'class_2013',
    'сегментация_eng': 'segmentation_eng',
    'локализация_производства': 'production_localization'
}

In [5]:
# Применяем перевод названий столбцов
df = df.rename(columns=translation_dict)

По какой-то причине в столбце с объёмом двигателя разделитель не поменялся на точку, сделаем замену методом replace.

In [6]:
# Преобразование столбца price_usd с заменой запятых на точки
df['engine_capacity_liters'] = df['engine_capacity_liters'].str.replace(',', '.')


In [7]:
#удалим лишние столбцы
df = df.drop(['modification', 'segment', 'dealer_center_name', 'payment_form', 'segmentation_eng', 'production_localization'], axis=1)

In [8]:
#посмотрим информацию о датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39966 entries, 0 to 39965
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    39966 non-null  int64  
 1   month                   39966 non-null  object 
 2   company                 39966 non-null  object 
 3   brand                   39966 non-null  object 
 4   model                   39966 non-null  object 
 5   manufacturing_year      39465 non-null  object 
 6   manufacturer_country    39966 non-null  object 
 7   fuel_type               36826 non-null  object 
 8   engine_capacity_liters  35708 non-null  object 
 9   transmission_type       36711 non-null  object 
 10  drive_type              35677 non-null  object 
 11  region                  39966 non-null  object 
 12  customer_type           32919 non-null  object 
 13  quantity                39960 non-null  float64
 14  price_usd               39966 non-null

Распишем смысл каждого столбца. Оценим важность каждого столбца для исследования.
* year – год продажи (2019)  
* month – месяц продажи (январь - сентябрь) - важный столбец  
* company – название автоцентра в котором продана машина - важный столбец, например, для оценки наиболее рентабельных автоцентров  
* brand – название продаваемой марки автомобиля - важный столбец, потому что необходимо знать, какие бренды чаще или реже продвются  
* model – название модели автомобиля - важный столбец, потому что необходимо знать, какие модели чаще или реже продвются  
* manufacturing_year – год производства автомобиля - важный столбец  
* manufacturer_country – страна, где произведен автомобиль - важный столбец  
* fuel_type – бензин, дизель, электричество, гибрид    
* engine_capacity_liters – объем двигателя автомобиля в литрах    
* transmission_type – тип коробки переключения передач   
* drive_type – тип привода   
* region – регион продажи - важный столбец, так как нам необходимо знать, в каких регионах лучше или хуже продаются авто  
* quantity – количество автомобилей в заказе - важный столбец, благодаря ему, например, можно посчитать среднее количество машин в 1 чеке  
* price_usd – цена автомобиля  - важный столбец, например, для сравнения цен на авто в разных автоцентрах  
* sale_usd – цена заказа (цена авто умноженная на количество и за вычетом скидок если есть)  - важный столбец, например, для подсчёта выручки или среднего чека  
* area – область продажи - важный столбец   
* segmentation_2013 – сегмент автомобиля,   
* class_2013 – класс автомобиля актуальный  

In [9]:
#посмотрим количество пропусков
df.isna().sum()

year                         0
month                        0
company                      0
brand                        0
model                        0
manufacturing_year         501
manufacturer_country         0
fuel_type                 3140
engine_capacity_liters    4258
transmission_type         3255
drive_type                4289
region                       0
customer_type             7047
quantity                     6
price_usd                    0
sale_usd                     0
area                         0
segmentation_2013            0
class_2013                   0
dtype: int64

In [10]:
#удалим строки, где в quantity пропуски
df=df.dropna(subset=['quantity'])
df.quantity.count()

39960

В столбце manufacturing_year немного пропусков 501, 
fuel_type 3140 пропусков, 
engine_capacity_liters 4258, 
transmission_type 3255 и drive_type 4289. Эти столбцы отражают характеристики авто, они не столь важны для анализа рынка, поэтому просто поставим на место пропусков заглушки в виде 0 для удобства работы.

In [11]:
#заменим пропуски
def fill_missing_values(df_to_fill, columns_to_fill):
    for column in columns_to_fill:
        df_to_fill[column] = df_to_fill[column].fillna('0')
    return df_to_fill

# Примените функцию к вашему DataFrame
columns_to_fill = ['manufacturing_year', 'fuel_type', 'engine_capacity_liters', 'transmission_type', 'drive_type','customer_type']
df = fill_missing_values(df, columns_to_fill)
df.isna().sum()

year                      0
month                     0
company                   0
brand                     0
model                     0
manufacturing_year        0
manufacturer_country      0
fuel_type                 0
engine_capacity_liters    0
transmission_type         0
drive_type                0
region                    0
customer_type             0
quantity                  0
price_usd                 0
sale_usd                  0
area                      0
segmentation_2013         0
class_2013                0
dtype: int64

In [12]:
# Цикл для удаления лишних пробелов в конце значений строковых столбцов
for column in df.columns:
    if df[column].dtype == "object":
        df[column] = df[column].str.replace(r' $', '', regex=True)

In [13]:
#функция для поиска неявных дублей
def translate_and_find_similar(df, columns_to_compare):
    def similar_values(value1, value2):
        value1_lower = unidecode(value1).lower()
        value2_lower = unidecode(value2).lower()
        return lev.distance(value1_lower, value2_lower) <= 2
    
    for column in columns_to_compare:
        if df[column].dtype == "object":
            unique_values = df[column].unique()
            num_unique_values = len(unique_values)
            
            for i in range(num_unique_values):
                for j in range(i + 1, num_unique_values):
                    value1, value2 = unique_values[i], unique_values[j]
                    if similar_values(value1, value2):
                        print(f"Similar values found in '{column}': '{value1}' and '{value2}'")

# Список столбцов, по которым нужно искать похожие значения
columns_to_compare = ['company','customer_type','area']

# Применение функции
print("Неявные дубликаты:")
translate_and_find_similar(df, columns_to_compare)

Неявные дубликаты:
Similar values found in 'company': 'Mercur Auto' and 'Mercur Autos'
Similar values found in 'company': 'Caspian Motors' and 'Каспиан Моторс'
Similar values found in 'company': 'Autokapital' and 'Автокапитал'
Similar values found in 'company': 'ММС Рус' and 'MMC RUS'
Similar values found in 'company': 'Равон Моторс Казахстан' and 'Ravon Motors Kazakstan'
Similar values found in 'customer_type': 'Физ. Лицо' and 'физ.лицо'
Similar values found in 'customer_type': 'Физ. Лицо' and 'ФизЛицо'
Similar values found in 'customer_type': 'Юр. Лицо' and 'Юр.Лицо'
Similar values found in 'customer_type': 'Юр. Лицо' and 'ЮрЛицо'
Similar values found in 'customer_type': 'Юр.Лицо' and 'ЮрЛицо'
Similar values found in 'customer_type': 'физ.лицо' and 'ФизЛицо'


In [14]:
# функция для просмотра уникальных значений в выбранных столбцах
for column in df.columns:
    unique_values = df[column].sort_values().unique()
    print(f"Уникальные значения в '{column}': {unique_values}\n")

Уникальные значения в 'year': [2019]

Уникальные значения в 'month': ['Август' 'Апрель' 'Июль' 'Июнь' 'Май' 'Март' 'Сентябрь' 'Февраль'
 'Январь']

Уникальные значения в 'company': ['Allur Auto' 'Almaty Motors Premium' 'Astana Motors' 'Autokapital'
 'Caspian Motors' 'Daewoo Bus Kazakhstan' 'Eurasia Motor Premium'
 'Hino Motors' 'Hyundai Com Trans Kazakhstan' 'MAN Truck & Bus Kazakhstan'
 'MMC RUS' 'Mercur Auto' 'Mercur Autos' 'Nissan Manufacturing RUS'
 'Ravon Motors Kazakstan' 'Renault Россия' 'Scandinavian Motors'
 'Scania Central Asia' 'Subaru Kazakhstan' 'TERRA MOTORS'
 'Toyota Motor Kazakhstan' 'Volkswagen Group Rus' 'Автодом Motors KST'
 'Автокапитал' 'Автомир ГК' 'Автомир-Центр' 'Автоцентр-Бавария'
 'БИПЭК АВТО' 'Вираж' 'Каспиан Моторс' 'Лифан Моторс Рус' 'ММС Рус'
 'Равон Моторс Казахстан' 'СВС-ТРАНС' 'СемАЗ' 'ТК КАМАЗ'
 'ТОО "Eurasia Motor Zhaik"' 'УзАвто-Казахстан' 'Хино Моторс Казахстан']

Уникальные значения в 'brand': ['ANKAI' 'Audi' 'BMW' 'Cadillac' 'Chevrolet' 'Daewoo' '

* Дубли в company: 'Mercur Auto' and 'Mercur Autos','Caspian Motors' and 'Каспиан Моторс', 'Autokapital' and 'Автокапитал', 'ММС Рус' and 'MMC RUS', 'Равон Моторс Казахстан' and 'Ravon Motors Kazakstan'
* В brand дублей нет
* В model: 3 и 3 серия, '6' '6 серия', в значении 'TQ-1/H-1' слиплись 2 вида моделей: Hyundai TQ-1 и Hyundai H-1.
* manufacturing_year есть 2 странных значения '2\xa0018' '2\xa0019'. Необходимо детальнее изучить столбец
* В manufacturer_country нет дубликатов
* fuel_type 1.6, 2 - эти значения не относятся к виду топлива, скорее всего в этот столбец попал объём двигателя, а значение 0 не может относиться ни к тому ни к другому. 
* engine_capacity_liters '#Н/Д', '0', '88 KWH', '400 Л.С.', 'AT', 'MT' - в этот столбец попали значения из других столбцов - мощность двигателя и тип коробки передач. Также есть неявные дубли '2' '2.0' '2.0h' '3' '3.0 L' '4' '4.0' '4.3' '4.3.' '4.98' '4.98 L.' '6.7' '6.7L'  Необходимо детальнее изучить столбец
* transmission_type '#Н/Д', '4WD', 'Передний' - значения из drive_type. Странные значения 'TDI' и '8', Необходимо детальнее изучить столбец и разбить на механику и автомат. 
* drive_type '#Н/Д' ('FWD', 'FF', 2 WD' '2WD'-передний) ('AWD' 'quattro' '4Motion' '4 WD''4WD'  '4X4'  '4x4'  '4х4'-полный) ('4X2' '4x2' '4х2' '4х2.2'-неполный) ('RWD' задний), попало значение 'Астана'. Необходимо детальнее изучить столбец и разбить на передний, задний, полный и неполный приводы.
* region нет дублей
* customer_type': 'Физ. Лицо' and 'физ.лицо', 'ФизЛицо', 'Юр. Лицо', 'Юр.Лицо', 'ЮрЛицо'
* В quantity значение '-1' - это возврат, а большое число проданных автомобилей 100 и 115 (возможно опт продажа таксопарку или на экспорт).
* price_usd - необходимо посмотреть выбросы (построить boxplot)
* В sale_usd странное значение -35588.25 и 0 - надо посмотреть количество таких значений и тоже проверить на выбросы
* В area всё впорядке
* segmentation_2013 норм
* В class_2013 всё норм

In [16]:
condition = (
    (df['fuel_type'].isin(['2', '1,6'])) |
    (df['engine_capacity_liters'].isin(['88 KWH', '400 Л.С.', 'AT', 'MT'])) |
    (df['transmission_type'].isin(['TDI', '8', '4WD', 'Передний'])) |
    (df['manufacturing_year'].isin(['2\xa0018', '2\xa0019'])) |
    (df['drive_type'] == 'Астана')
)

# Подсчет количества удаляемых строк
count_deleted = len(df[condition])

# Вывод количества удаляемых строк
print(f'Будет удалено строк: {count_deleted}')



Удалено строк: 1319


In [ ]:
# Удаление строк
df = df[~condition]
#проверим результат
print(39960 - df['model'].count())

Теперь объединим год с месяцем в дату продажи

In [21]:
import calendar

#преобразуем названия месяцев в числа
def monthToNum(shortMonth):
    return {
            'Январь': 1,
            'Февраль': 2,
            'Март': 3,
            'Апрель': 4,
            'Май': 5,
            'Июнь': 6,
            'Июль': 7,
            'Август': 8,
            'Сентябрь': 9, 
            'Октябрь': 10,
            'Ноябрь': 11,
            'Декабрь': 12
    }[shortMonth]
# Применяем функцию к столбцу 'month' и создаем новый столбец 'number_month'
df['number_month'] = df['month'].apply(monthToNum)
#преобразуем месяц и год в строковые значения для объединения в один столбец
df['year'] = df['year'].astype('str')
df['number_month'] = df['number_month'].astype('str')
# Объединяем год и месяц с разделителем '-'
df['date'] = df['year'] + '-' + df['number_month']
# Преобразовываем столбец 'date' в тип данных datetime
df['date'] = pd.to_datetime(df['date'], format='%Y-%m')
#проверим изменения 
df['date'].head()

0   2019-05-01
1   2019-08-01
2   2019-04-01
3   2019-07-01
4   2019-07-01
Name: date, dtype: datetime64[ns]

In [22]:
#df['engine_capacity_liters']=df['engine_capacity_liters'].astype('float')

что нужно сделать со столбцом engine_capacity_liters?

In [23]:
#df['year']=df['year'].astype('int')
#df['month']=df['month'].astype('string')
#df['brand']=df['brand'].astype('string')
def type_str(df, value):
    df[value] = df[value].astype('int')
    return df
s = ['year','number_month', 'manufacturing_year','quantity']
df = type_str(df, s)

In [24]:
def type_str(df, value):
    df[value] = df[value].astype('string')
    return df
s = ['month', 'company', 'model', 'brand','manufacturer_country','transmission_type','region','customer_type','area']
df = type_str(df, s)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38641 entries, 0 to 39965
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   year                    38641 non-null  int32         
 1   month                   38641 non-null  string        
 2   company                 38641 non-null  string        
 3   brand                   38641 non-null  string        
 4   model                   38641 non-null  string        
 5   manufacturing_year      38641 non-null  int32         
 6   manufacturer_country    38641 non-null  string        
 7   fuel_type               38641 non-null  object        
 8   engine_capacity_liters  38641 non-null  object        
 9   transmission_type       38641 non-null  string        
 10  drive_type              38641 non-null  object        
 11  region                  38641 non-null  string        
 12  customer_type           38641 non-null  string

*кодируем страны в столбце страна производства в alpha3 (RUS, KAZ, USA и т.д.)
*кодируем тип топлива в (F - fuel, D - diesel, E - electro, HYB - hybrid или по желанию студента)
создадим из столбцов год и месяц единый столбец дата продажи (число месяца ставим
последний день - 30 или 31)
удаляем неявные дубликаты в столбце название компании (названия на английском и русском)
приведем в порядок столбец объем двигателя (заменим запятую на точку, уберем букву L,
разберемся с мотором 400 литров у ягуара (там перепутано объем и мощность двигателя - тем
кто это найдет и определит нормальный литраж двигателя можно значок давать за въедливость,
а можно просто удалить)) + есть неправильные объемы двигателя для Шевроле Нива - 21.6, 20.6
и так далее
столбцы регион и область - привести к нижнему регистру с заглавной буквы
столбец тип трансмиссии - убрать неявные дубликаты (для простоты оставим два типа
трансмиссии: механика и автомат)
удалим столбцы месяц и год
дата продажи к типу дата
столбцы тип топлива, тип трансмиссии, тип привода, сегмент 2013 и класс 2013 можно
перевести в категориальные

## Исследовательский анализ

In [26]:
print("Количество дубликатов:", df.duplicated().sum())
print("Количество дубликатов в процентах:", round(df.duplicated().sum()/df['year'].count()*100, 2), "%")

Количество дубликатов: 21420
Количество дубликатов в процентах: 55.43 %


Большое количество дубликатов. 

In [30]:
#df.duplicated(keep=False)